# Лабораторна робота 1 з "Методів криптоаналізу"
## Тема: Баєсiвський пiдхiд в криптоаналiзi: побудова i дослiдження детермiнiстичної та стохастичної вирiшуючих функцiй.

**Виконали**\
Дигас Богдан, ФІ-52мн\
Юрчук Олексій, ФІ-52мн

In [1]:
import pandas as pd
import os

In [2]:
def visualize_probability_data(filepath: str):
    if not os.path.exists(filepath):
        print(f"Error: The file '{filepath}' was not found.")
        return

    try:
        df = pd.read_csv(filepath, nrows=2, header=None)
        df.index = ['Opened Texts Probabilities (M)', 'Keys Probabilities (K)']
        df.columns = list(range(len(df.columns)))

        print("Probability Data Table:")
        print(df)

    except Exception as e:
        print(f"An error occurred while processing the file: {e}")

In [3]:
file_path = os.path.join('.', 'var_data', 'prob_15.csv')
    
visualize_probability_data(file_path)

Probability Data Table:
                                  0     1     2     3     4     5     6   \
Opened Texts Probabilities (M)  0.11  0.11  0.11  0.11  0.11  0.03  0.03   
Keys Probabilities (K)          0.14  0.14  0.04  0.04  0.04  0.04  0.04   

                                  7     8     9     10    11    12    13  \
Opened Texts Probabilities (M)  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
Keys Probabilities (K)          0.04  0.04  0.04  0.04  0.04  0.04  0.04   

                                  14    15    16    17    18    19  
Opened Texts Probabilities (M)  0.03  0.03  0.03  0.03  0.03  0.03  
Keys Probabilities (K)          0.04  0.04  0.04  0.04  0.04  0.04  


In [4]:
def visualize_cipher_table(filepath: str):
    if not os.path.exists(filepath):
        print(f"Error: The file '{filepath}' was not found.")
        return

    try:
        df = pd.read_csv(filepath, header=None)

        row_names = [f"M_{i}" for i in range(df.shape[0])]
        col_names = [f"K_{i}" for i in range(df.shape[1])]
        df.index = row_names
        df.columns = col_names

        print("Cipher table:")
        print(df)

    except Exception as e:
        print(f"An error occurred while processing the file: {e}")

In [5]:
file_path = os.path.join('.', 'var_data', 'table_15.csv')
visualize_cipher_table(file_path)

Cipher table:
      K_0  K_1  K_2  K_3  K_4  K_5  K_6  K_7  K_8  K_9  K_10  K_11  K_12  \
M_0     2   14   17   12    3   15    4    5    0    6     9    16     1   
M_1    11   17    4    8   16    3   12   15    0   14     6    19     5   
M_2    13   17   15   11    9   10    2    8   18   14     6    19     3   
M_3    15    9   17   13   19   11   12    2    3   18     1    14     4   
M_4     5    7    4    2   17   13    1   15   12    0     6    14     3   
M_5     8   16    6   11    3   17   14   10   19    7     5    18     4   
M_6     2    6   13   12    1    7    5    0   15   10     8     3     9   
M_7    19   15    4    8   17   18   16    6   10    7     2     1     3   
M_8    13    8   18    4   17    3    1   16   10    5     9    19     2   
M_9     9    6   15   13    0    1   12    2   19   14    18     4     3   
M_10    5   19   12    1    7   17    8    2   16   13     4    14    18   
M_11   11   18   10    9   15    6   13   17    1    7     8     5    19  

In [6]:
prob_file = os.path.join('.', 'var_data', 'prob_15.csv')
table_file = os.path.join('.', 'var_data', 'table_15.csv')

In [7]:
from typing import Tuple
from decimal import Decimal, getcontext, ROUND_HALF_UP

In [8]:
def calculate_ciphertext_probabilities_decimal(
    prob_filepath: str,
    table_filepath: str,
    rounding_digits: int = 7
) -> Tuple[pd.Series, pd.Series]:
    """
    Read probability file (first two rows: P(M) then P(K)) and a cipher mapping table.
    Returns (p_plaintexts_series, p_ciphertexts_series) with values rounded to `rounding_digits`.
    """

    # safety checks
    if not os.path.exists(prob_filepath):
        raise FileNotFoundError(f"Probability file not found: {prob_filepath}")
    if not os.path.exists(table_filepath):
        raise FileNotFoundError(f"Cipher table file not found: {table_filepath}")

    # Set Decimal precision sufficiently high for multiplications
    getcontext().prec = 50

    # --- Read probability table (first two rows) ---
    prob_df = pd.read_csv(prob_filepath, nrows=2, header=None)
    # Convert to Decimal list (use str() to avoid float->Decimal surprises)
    p_m_list = [Decimal(str(v)) for v in prob_df.iloc[0].tolist()]
    p_k_list = [Decimal(str(v)) for v in prob_df.iloc[1].tolist()]

    # --- Read cipher mapping table ---
    cipher_df = pd.read_csv(table_filepath, header=None)
    cipher_np = cipher_df.to_numpy(dtype=int)
    max_cipher_index = int(cipher_np.max())

    # Initialize P(C) with Decimal zeros
    p_c = [Decimal('0') for _ in range(max_cipher_index + 1)]

    # --- Accumulate with Decimal precision ---
    rows = cipher_df.shape[0]
    cols = cipher_df.shape[1]
    for i in range(rows):            # plaintext index i -> M_i
        for j in range(cols):        # key index j -> K_j
            c_idx = int(cipher_df.iat[i, j])
            p_c[c_idx] += p_m_list[i] * p_k_list[j]

    # --- Normalize (protect against tiny numeric drift) ---
    total = sum(p_c)
    if total == Decimal('0'):
        raise ValueError("Computed total probability is zero.")
    # scale so sum == 1
    p_c = [pc / total for pc in p_c]

    # --- Round / quantize to requested digits ---
    quant_str = '1e-' + str(rounding_digits)
    quant = Decimal(quant_str)
    p_c_rounded = [pc.quantize(quant, rounding=ROUND_HALF_UP) for pc in p_c]
    p_m_rounded = [pm.quantize(quant, rounding=ROUND_HALF_UP) for pm in p_m_list]

    # Re-normalize after quantize to ensure summed rounding is consistent (optional)
    # If you want the final rounded P(C) to sum exactly to 1, we can adjust the largest element slightly.
    sum_rounded = sum(p_c_rounded)
    if sum_rounded != Decimal('1'):
        diff = Decimal('1') - sum_rounded
        # add diff to the largest P(C) to keep things consistent
        max_idx = int(pd.Series(p_c_rounded).idxmax())
        p_c_rounded[max_idx] = (p_c_rounded[max_idx] + diff).quantize(quant, rounding=ROUND_HALF_UP)

    # Build pandas Series with readable labels
    p_plaintexts = pd.Series(
        [float(x) for x in p_m_rounded],
        index=[f"M_{i}" for i in range(len(p_m_rounded))]
    )
    p_ciphertexts = pd.Series(
        [float(x) for x in p_c_rounded],
        index=[f"C_{i}" for i in range(len(p_c_rounded))]
    )

    return p_plaintexts, p_ciphertexts

ТОЧНІСТЬ ДО P(C) додати

In [9]:
p_plaintexts, p_ciphertexts = calculate_ciphertext_probabilities_decimal(prob_file, table_file, rounding_digits=7)

if not p_plaintexts.empty and not p_ciphertexts.empty:
    print("Розраховані ймовірності P(C) для кожного шифротексту:")
    print(p_ciphertexts)
    print("\nСума всіх ймовірностей P(C) має дорівнювати 1:")
    print(f"Сума: {p_ciphertexts.sum():.7f}")

Розраховані ймовірності P(C) для кожного шифротексту:
C_0     0.040
C_1     0.043
C_2     0.054
C_3     0.040
C_4     0.043
C_5     0.057
C_6     0.052
C_7     0.051
C_8     0.049
C_9     0.054
C_10    0.040
C_11    0.057
C_12    0.040
C_13    0.057
C_14    0.051
C_15    0.057
C_16    0.049
C_17    0.068
C_18    0.046
C_19    0.052
dtype: float64

Сума всіх ймовірностей P(C) має дорівнювати 1:
Сума: 1.0000000


In [10]:
def calculate_joint_probabilities(p_plaintexts: pd.Series, p_ciphertexts: pd.Series) -> pd.DataFrame:
    if p_plaintexts.empty or p_ciphertexts.empty:
        print("Помилка: Вхідні дані для обчислення P(M, C) відсутні.")
        return pd.DataFrame()

    joint_prob_df = pd.DataFrame(index=p_plaintexts.index, columns=p_ciphertexts.index)
    
    for m_idx in p_plaintexts.index:
        for c_idx in p_ciphertexts.index:
            p_m = p_plaintexts.loc[m_idx]
            p_c = p_ciphertexts.loc[c_idx]
            joint_prob_df.loc[m_idx, c_idx] = p_m * p_c # P(M_i, C_j) = P(M_i) * P(C_j) - independence

    joint_prob_df.index = [f"M_{i}" for i in joint_prob_df.index]
    joint_prob_df.columns = [f"C_{int(i)}" for i in joint_prob_df.columns]

    return joint_prob_df

In [11]:
prob_file = os.path.join('.', 'var_data', 'prob_15.csv')
table_file = os.path.join('.', 'var_data', 'table_15.csv')

p_plaintexts, p_ciphertexts = calculate_ciphertext_probabilities(prob_file, table_file)
    
if not p_plaintexts.empty and not p_ciphertexts.empty:
    joint_probabilities = calculate_joint_probabilities(p_plaintexts, p_ciphertexts)
        
    if not joint_probabilities.empty:
        print("Таблиця спільних ймовірностей P(M, C):")
        print(joint_probabilities)
        print("\nСума всіх ймовірностей P(М, C) має дорівнювати 1:")
        print(f"Сума: {joint_probabilities.values.sum():.6f}")

NameError: name 'calculate_ciphertext_probabilities' is not defined

In [ ]:
def calculate_conditional_probabilities(joint_probabilities: pd.DataFrame, p_ciphertexts: pd.Series) -> pd.DataFrame:
    if joint_probabilities.empty or p_ciphertexts.empty:
        print("Помилка: Вхідні дані для обчислення P(M|C) відсутні.")
        return pd.DataFrame()

    conditional_prob_df = joint_probabilities.copy()

    for c_idx in p_ciphertexts.index:

        col_name = f"C_{int(c_idx)}"
        p_c = p_ciphertexts.loc[c_idx]

        if p_c > 0:
            conditional_prob_df.loc[:, col_name] = joint_probabilities.loc[:, col_name] / p_c
        else:
            conditional_prob_df.loc[:, col_name] = 0

    return conditional_prob_df

In [ ]:
conditional_probabilities = calculate_conditional_probabilities(joint_probabilities, p_ciphertexts)
            
if not conditional_probabilities.empty:
    print("Таблиця умовних ймовірностей P(M|C):")
    print(conditional_probabilities)
    
    print("\nПеревірка суми стовпців (кожен стовпець має дорівнювати 1):")
    column_sums = conditional_probabilities.sum(axis=0)
    print(column_sums)

Таблиця умовних ймовірностей P(M|C):
       C_0   C_1   C_2   C_3   C_4   C_5   C_6   C_7   C_8   C_9  C_10  C_11  \
M_0   0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11   
M_1   0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11   
M_2   0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11   
M_3   0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11   
M_4   0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11  0.11   
M_5   0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
M_6   0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
M_7   0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
M_8   0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
M_9   0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
M_10  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
M_1

In [ ]:
def find_optimal_decision_function(conditional_probabilities: pd.DataFrame) -> pd.Series: 
    if conditional_probabilities.empty:
        print("Помилка: Вхідні дані для визначення детермінованої функції відсутні.")
        return pd.Series(dtype=str)

    optimal_decisions = conditional_probabilities.idxmax(axis=1) # \delta(C_D) = max(P(M_i | C_D))

    return optimal_decisions

In [ ]:
if not conditional_probabilities.empty:
    optimal_decisions = find_optimal_decision_function(conditional_probabilities)
    if not optimal_decisions.empty:
        print("Оптимальна детермінована функція рішення δ(C):")
        print(optimal_decisions)

Оптимальна детермінована функція рішення δ(C):
M_0     C_18
M_1     C_18
M_2     C_18
M_3     C_18
M_4     C_18
M_5      C_6
M_6      C_6
M_7      C_6
M_8      C_6
M_9      C_6
M_10     C_6
M_11     C_6
M_12     C_6
M_13     C_6
M_14     C_6
M_15     C_6
M_16     C_6
M_17     C_6
M_18     C_6
M_19     C_6
dtype: object


In [ ]:
def calculate_stochastic_function(p_plaintexts: pd.Series, p_ciphertexts: pd.Series) -> pd.DataFrame:
    if p_plaintexts.empty or p_ciphertexts.empty:
        print("Помилка: Вхідні дані для побудови стохастичної функції відсутні.")
        return pd.DataFrame()

    num_plaintexts = len(p_plaintexts)    
    uniform_prob = 1.0 / num_plaintexts # Format 
    
    stochastic_matrix = pd.DataFrame(
        uniform_prob, 
        index=p_plaintexts.index, 
        columns=p_ciphertexts.index
    )

    stochastic_matrix.index = [f"M_{i}" for i in stochastic_matrix.index]
    stochastic_matrix.columns = [f"C_{int(i)}" for i in stochastic_matrix.columns]

    return stochastic_matrix

In [ ]:
stochastic_function = calculate_stochastic_function(p_plaintexts, p_ciphertexts)
if not stochastic_function.empty:
    print("Матриця стохастичної функції рішення δ_S:")
    print(stochastic_function)
    
    print("\nПеревірка суми рядків (кожен рядок має дорівнювати 1):")
    row_sums = stochastic_function.sum(axis=1)
    print(row_sums)

Матриця стохастичної функції рішення δ_S:
       C_0   C_1   C_2   C_3   C_4   C_5   C_6   C_7   C_8   C_9  C_10  C_11  \
M_0   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_1   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_2   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_3   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_4   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_5   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_6   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_7   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_8   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_9   0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
M_10  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  

---

In [ ]:
def extract_optimal_probabilities(optimal_decisions: pd.Series, conditional_probabilities: pd.DataFrame) -> pd.Series:
    if optimal_decisions.empty or conditional_probabilities.empty:
        print("Помилка: Вхідні дані для виділення оптимальних ймовірностей відсутні.")
        return pd.Series(dtype=float)
        
    optimal_probabilities = pd.Series(index=optimal_decisions.index, dtype=float)

    for c_label, m_label in optimal_decisions.items():
        try:
            optimal_probabilities.loc[c_label] = conditional_probabilities.loc[m_label, c_label]
        except KeyError:
            print(f"Помилка: Не вдалося знайти ймовірність для M='{m_label}', C='{c_label}'.")

    return optimal_probabilities

In [ ]:
extracted_probs = extract_optimal_probabilities(optimal_decisions, conditional_probabilities)
print("\nУмовні ймовірності для оптимальних рішень:")
print(extracted_probs)

Помилка: Не вдалося знайти ймовірність для M='C_18', C='M_0'.
Помилка: Не вдалося знайти ймовірність для M='C_18', C='M_1'.
Помилка: Не вдалося знайти ймовірність для M='C_18', C='M_2'.
Помилка: Не вдалося знайти ймовірність для M='C_18', C='M_3'.
Помилка: Не вдалося знайти ймовірність для M='C_18', C='M_4'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_5'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_6'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_7'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_8'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_9'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_10'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_11'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_12'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_13'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_14'.
Помилка: Не вдалося знайти ймовірність для M='C_6', C='M_15'.
Помилка: Не в

In [ ]:
def count_optimal_plaintexts(conditional_probabilities: pd.DataFrame) -> pd.Series:
    if conditional_probabilities.empty:
        print("Помилка: Вхідна таблиця умовних ймовірностей порожня.")
        return pd.Series(dtype=int)
    
    max_probs = conditional_probabilities.max(axis=0)
    optimal_count = (conditional_probabilities == max_probs).sum(axis=0)
    
    return optimal_count

In [ ]:
optimal_counts = count_optimal_plaintexts(conditional_probabilities)
print("\nКількість відкритих текстів з максимальною умовною ймовірністю:")
print(optimal_counts)


Кількість відкритих текстів з максимальною умовною ймовірністю:
C_0     5
C_1     5
C_2     5
C_3     5
C_4     5
C_5     5
C_6     5
C_7     5
C_8     5
C_9     5
C_10    5
C_11    5
C_12    5
C_13    5
C_14    5
C_15    5
C_16    5
C_17    5
C_18    5
C_19    5
dtype: int64


In [ ]:
def create_improved_stochastic_function(conditional_probabilities: pd.DataFrame) -> pd.DataFrame:
    if conditional_probabilities.empty:
        print("Помилка: Вхідна таблиця умовних ймовірностей порожня.")
        return pd.DataFrame()
    
    improved_matrix = pd.DataFrame(0.0, index=conditional_probabilities.index, columns=conditional_probabilities.columns)

    for column in conditional_probabilities.columns:
        max_prob = conditional_probabilities[column].max()
        
        if max_prob > 0: # Find all rows (plaintexts) that have chosen maximum probability
            optimal_plaintexts = conditional_probabilities[conditional_probabilities[column] == max_prob].index 
            num_optimal = len(optimal_plaintexts)
            
            if num_optimal > 0:
                probability_to_assign = 1.0 / num_optimal
                improved_matrix.loc[optimal_plaintexts, column] = probability_to_assign
    
    return improved_matrix

In [ ]:
improved_stochastic_function = create_improved_stochastic_function(conditional_probabilities)
print("Покращена стохастична функція:")
print(improved_stochastic_function)

Покращена стохастична функція:
      C_0  C_1  C_2  C_3  C_4  C_5  C_6  C_7  C_8  C_9  C_10  C_11  C_12  \
M_0   0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2   0.2   0.2   0.2   
M_1   0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2   0.2   0.2   0.2   
M_2   0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2   0.2   0.2   0.2   
M_3   0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2   0.2   0.2   0.2   
M_4   0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2   0.2   0.2   0.2   
M_5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
M_6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
M_7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
M_8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
M_9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
M_10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
M_11  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

In [ ]:
def calculate_average_loss_deterministic(joint_probabilities: pd.DataFrame, optimal_decisions: pd.Series) -> float:
    if joint_probabilities.empty or optimal_decisions.empty:
        print("Помилка: Вхідні дані для розрахунку середніх втрат відсутні.")
        return 0.0

    total_correct_prob = 0.0
    
    for m_label, c_label in optimal_decisions.items():
        total_correct_prob += joint_probabilities.loc[m_label, c_label]
                
    return 1.0 - total_correct_prob

In [ ]:
avg_loss = calculate_average_loss_deterministic(joint_probabilities, optimal_decisions)
print(f"\nСередні втрати для детермінованої функції: {avg_loss:.9f}")


Середні втрати для детермінованої функції: 0.951300000


In [ ]:
def calculate_average_loss_stochastic(joint_probabilities: pd.DataFrame, stochastic_function: pd.DataFrame) -> float:
    # \sum_{C_h} \sum_{M_i} P(M_i \mid C_h) \left( \sum_{M_j \neq M_i} s(C_h, M_j) \right)
    if joint_probabilities.empty or stochastic_function.empty:
        print("Помилка: Вхідні дані для розрахунку середніх втрат відсутні.")
        return 0.0
    
    loss_matrix = 1 - stochastic_function
    total_loss_matrix = joint_probabilities * loss_matrix
    
    average_loss = total_loss_matrix.values.sum()
    
    return average_loss

In [ ]:
avg_loss_stoch = calculate_average_loss_stochastic(joint_probabilities, improved_stochastic_function)
print(f"\nСередні втрати для стохастичної функції: {avg_loss_stoch:.9f}")


Середні втрати для стохастичної функції: 0.890000000
